In [1]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [2]:
print("Total Sentences:", len(text_pairs))
#text_pairs

Total Sentences: 20


In [5]:
import random
for i in range(5):
    print(random.choice(text_pairs));

('Nope.', '[start] না. [end]')
('About hundrad million people speaks in Bangla.', '[start] প্রায় এক কোটি মানুষ বাংলায় কথা বলে. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')


In [13]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.20 * total_data_length)

In [14]:
print(total_data_length)
print(num_val_samples)

20
4


In [15]:
num_train_samples = (total_data_length - 2 * num_val_samples)
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [18]:
print(f"Number of Train Pairs: {len(train_pairs)}")
print(f"Number of Val Pairs: {len(val_pairs)}")
print(f"Number of Test Pairs: {len(test_pairs)}")

Number of Train Pairs: 12
Number of Val Pairs: 4
Number of Test Pairs: 4


In [20]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + ""
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [21]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~]'

In [22]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from tensorflow.keras import layers

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return(tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    ))

vocab_size = 10000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

In [23]:
train_english_texts = [pair[0] for pair in train_pairs]
train_bangla_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_bangla_texts)

In [24]:
batch_size = 64

def format_dataset(eng, ban):
    eng = source_vectorization(eng)
    ban = source_vectorization(ban)
    return({"english": eng, "bangla": ban[:, :-1]}, ban[:, 1:])

def make_dataset(pairs):
    eng_texts, ban_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ban_texts = list(ban_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ban_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return(dataset.shuffle(2048).prefetch(16).cache())

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [22]:
#print(list(train_ds.as_numpy_iterator()));

In [25]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['bangla'].shape: {inputs['bangla'].shape}")
    print(f"target.shape: {targets.shape}")

inputs['english'].shape: (12, 20)
inputs['bangla'].shape: (12, 19)
target.shape: (12, 19)


In [26]:
from tensorflow import keras

embed_dim = 256
latent_dim = 1024

In [27]:
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [44]:
past_target = keras.Input(shape=(None,), dtype="int64", name="bangla")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

KerasTensor(type_spec=TensorSpec(shape=(None, None, 1024), dtype=tf.float32, name=None), name='gru_2/PartitionedCall:1', description="created by layer 'gru_2'")


In [43]:
#seq2seq_rnn.summary()

In [29]:
seq2seq_rnn.compile(
    optimizer = "rmsprop",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)
seq2seq_rnn.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
1/1 [==============================] - 24s 24s/step - loss: 2.7861 - accuracy: 0.0000e+00 - val_loss: 2.6736 - val_accuracy: 0.8261
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 2.6672 - accuracy: 0.8261 - val_loss: 1.4697 - val_accuracy: 0.8261
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 1.3229 - accuracy: 0.8261 - val_loss: 0.3594 - val_accuracy: 0.8261
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 0.3502 - accuracy: 0.8261 - val_loss: 0.6707 - val_accuracy: 0.2609
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 0.6368 - accuracy: 0.3188 - val_loss: 0.3906 - val_accuracy: 0.8261
Epoch 6/50
1/1 [==============================] - 2s 2s/step - loss: 0.3333 - accuracy: 0.8261 - val_loss: 0.3044 - val_accuracy: 0.8261
Epoch 7/50
1/1 [==============================] - 2s 2s/step - loss: 0.2346 - accuracy: 0.8261 - val_loss: 0.2777 - val_accuracy: 0.8261
Epoch 8/50
1/1 [===================

In [30]:
import numpy as np
ban_vocab = target_vectorization.get_vocabulary()
ban_index_lookup = dict(zip(range(len(ban_vocab)), ban_vocab))
max_decoded_sentence_length = 20

In [31]:
# English texts
train_eng_texts = [pair[0] for pair in train_pairs]
print(f"Train Eng Text: {len(train_eng_texts)}")

test_eng_texts = [pair[0] for pair in test_pairs]
print(f"Test Eng Text: {len(test_eng_texts)}")

val_eng_texts = [pair[0] for pair in val_pairs]
print(f"Val Eng Text: {len(val_eng_texts)}")

# Bangla texts
train_ban_texts = [pair[1] for pair in train_pairs]
print(f"\nTrain Ban Text: {len(train_ban_texts)}")

test_ban_texts = [pair[1] for pair in test_pairs]
print(f"Test Ban Text: {len(test_ban_texts)}")

val_ban_texts = [pair[1] for pair in val_pairs]
print(f"Val Ban Text: {len(val_ban_texts)}")

Train Eng Text: 12
Test Eng Text: 4
Val Eng Text: 4

Train Ban Text: 12
Test Ban Text: 4
Val Ban Text: 4


In [32]:
input_sentence = random.choice(test_eng_texts)
print(input_sentence)
tokenized_input_sentence = source_vectorization([input_sentence])
print(tokenized_input_sentence)

Bangla is a language.
tf.Tensor([[2 3 4 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)


In [41]:
input_sentence = random.choice(test_pairs)
print(input_sentence[0])
print(input_sentence[1])
tokenized_input_sentence0 = source_vectorization([input_sentence[0]])
print(tokenized_input_sentence0)
tokenized_input_sentence1 = source_vectorization([input_sentence[1]])
print(tokenized_input_sentence1)

I do not know him.
[start] আমি উনাকে চিনি না. [end]
tf.Tensor([[7 1 1 6 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)


In [45]:
decoded_sentence = "[start]"
tokenized_target_sentence = target_vectorization(decoded_sentence)
print(tokenized_target_sentence)
next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
print(next_token_predictions[0, 0, :].shape)
print(next_token_predictions[0, 0, :])
sampled_token_index = np.argmax(next_token_predictions[0, 0, :]);
print(sampled_token_index)
sampled_token = ban_index_lookup[sampled_token_index]
print(sampled_token)
decoded_sentence += " " + sampled_token
print(decoded_sentence);

tokenized_target_sentence = target_vectorization([decoded_sentence])
print(tokenized_target_sentence)
next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
print(next_token_predictions[0, 1, :].shape)
print(next_token_predictions[0, 1, :])
sampled_token_index = np.argmax(next_token_predictions[0, 1, :]);
print(sampled_token_index)
sampled_token = ban_index_lookup[sampled_token_index]
print(sampled_token)
decoded_sentence += " " + sampled_token
print(decoded_sentence);

tf.Tensor([2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(21,), dtype=int64)


ValueError: Data cardinality is ambiguous:
  x sizes: 1, 21
Make sure all arrays contain the same number of samples.